#### LongContextReorder
Propósito del ejemplo <br>
Se demuestra el uso del retriever LongContextReorderRetriever. Este retriever pertenece a la familia de Rerankers. El objetivo de estos modelos es reordenar los documentos extraidos por un retriever de mayor a menor relevancia. 

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

modelo_embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
textos = [
    """Un eclipse de Sol es un fenómeno astronómico.""",
    """Un eclipse puede producir ceguera.""",
    """Se dice que con la Luna llena cambia el tiempo.""",
    """Un eclipse se produce cuando la Luna se interpone entre la Tierra y el Sol.""",
    """El eclipse más antiguo registrado ocurrió en China.""",
    """La astronomía estudia el movimiento del Sol y la Luna.""",
    """El Sol es esencial para la vida en la Tierra.""",
    """La Luna siempre ha tenido un encanto poético.""",
    """De chico quería ser astrónomo.""",
    """Anoche hubo Luna llena.""",
    """Mañana va a ser un día soleado."""
] 
# Crear un retriever
persist_directory="chroma2"
chroma_db = Chroma.from_texts(textos, embedding=modelo_embedding,persist_directory=persist_directory)
retriever = chroma_db.as_retriever(search_kwargs={"k": 11})

# Crear la consulta
query = "¿Qué sabes de eclipses?"

# Recuperar documentos en orden de relevancia
docs = retriever.invoke(query)
docs

[Document(page_content='Un eclipse de Sol es un fenómeno astronómico.'),
 Document(page_content='Un eclipse se produce cuando la Luna se interpone entre la Tierra y el Sol.'),
 Document(page_content='Un eclipse puede producir ceguera.'),
 Document(page_content='De chico quería ser astrónomo.'),
 Document(page_content='El eclipse más antiguo registrado ocurrió en China.'),
 Document(page_content='La astronomía estudia el movimiento del Sol y la Luna.'),
 Document(page_content='Se dice que con la Luna llena cambia el tiempo.'),
 Document(page_content='La Luna siempre ha tenido un encanto poético.'),
 Document(page_content='El Sol es esencial para la vida en la Tierra.'),
 Document(page_content='Anoche hubo Luna llena.'),
 Document(page_content='Mañana va a ser un día soleado.')]

In [6]:
# Reordenar los documentos
from langchain_community.document_transformers import LongContextReorder

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirma que los documentos mas relevantes se ubican al principio y al final de la lista.
reordered_docs

[Document(page_content='Un eclipse de Sol es un fenómeno astronómico.'),
 Document(page_content='Un eclipse puede producir ceguera.'),
 Document(page_content='El eclipse más antiguo registrado ocurrió en China.'),
 Document(page_content='Se dice que con la Luna llena cambia el tiempo.'),
 Document(page_content='El Sol es esencial para la vida en la Tierra.'),
 Document(page_content='Mañana va a ser un día soleado.'),
 Document(page_content='Anoche hubo Luna llena.'),
 Document(page_content='La Luna siempre ha tenido un encanto poético.'),
 Document(page_content='La astronomía estudia el movimiento del Sol y la Luna.'),
 Document(page_content='De chico quería ser astrónomo.'),
 Document(page_content='Un eclipse se produce cuando la Luna se interpone entre la Tierra y el Sol.')]

##### Codigo alternativo para limpiar la BD

In [ ]:
# Contar el numero de documentos en la BD
n = chroma_db._collection.count()

# Eliminarlos
chroma_db.delete_collection()

# Verificar si la coleccion existe. Si ha sido borrada por completo ejecutar
#    la clausula except.
try:
    chroma_db._collection.count()
except StopIteration:
    print(f"{n} items han sido eliminados de la coleccion.")